<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Causal_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Basic Data Loading and Cleaning

In [206]:
#loading pre-cleaned csv files
clean_plaintiff_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_RnD.csv')
clean_plaintiff_RnD = clean_plaintiff_RnD[['case_number', 'filing_quarter_rnd',
                                           'closing_quarter_rnd', 'delta_rnd_sales', 'minus_one_rnd',
                                           'minus_two_rnd', 'minus_three_rnd',
                                           'plus_one_rnd', 'plus_two_rnd',
                                           'plus_three_rnd', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]

clean_defendent_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_RnD.csv')
clean_defendent_RnD = clean_defendent_RnD[['case_number', 'filing_quarter_rnd',
                                           'closing_quarter_rnd','delta_rnd_sales', 'minus_one_rnd',
                                           'minus_two_rnd', 'minus_three_rnd',
                                           'plus_one_rnd', 'plus_two_rnd',
                                           'plus_three_rnd', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]



clean_plaintiff_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_npm.csv')
clean_plaintiff_npm = clean_plaintiff_npm[['case_number','filing_quarter_npm',
                                           'closing_quarter_npm', 'delta_npm', 'minus_one_npm',
                                           'minus_two_npm', 'minus_three_npm',
                                           'plus_one_npm', 'plus_two_npm',
                                           'plus_three_npm', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]

clean_defendent_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_npm.csv')
clean_defendent_npm = clean_defendent_npm[['case_number', 'filing_quarter_npm',
                                           'closing_quarter_npm', 'delta_npm', 'minus_one_npm',
                                           'minus_two_npm', 'minus_three_npm',
                                           'plus_one_npm', 'plus_two_npm',
                                           'plus_three_npm', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]


pd.set_option('display.max_columns', None)

In [207]:
#adding defendant/plaintiff dummies and cocat merging into single df
clean_plaintiff_npm['Plaintiff'] = 1
clean_defendent_npm['Defendant'] = 1

npm = pd.concat([clean_defendent_npm, clean_plaintiff_npm], axis=0)
npm['Defendant'] = npm['Defendant'].fillna(0)
npm['Plaintiff'] = npm['Plaintiff'].fillna(0)


clean_plaintiff_RnD['Plaintiff'] = 1
clean_defendent_RnD['Defendant'] = 1

rnd = pd.concat([clean_defendent_RnD, clean_plaintiff_RnD], axis=0)
rnd['Defendant'] = rnd['Defendant'].fillna(0)
rnd['Plaintiff'] = rnd['Plaintiff'].fillna(0)

In [208]:
JFC_data = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/without_dummies_cases.csv')

JFC_data = JFC_data[['case_number', 'case_name','PLT', 'DEF', 'jurisdictional_basis',
                     'CIRCUIT', 'DISTRICT', 'OFFICE', 'JURY', 'NOS', 'CLASSACT',
                     'COUNTY', 'PROCPROG', 'DISP', 'NOJ', 'JUDGMENT']]

In [209]:
npm = npm.merge(JFC_data, on='case_number', how='left')
rnd = rnd.merge(JFC_data, on='case_number', how='left')

In [210]:
npm.head()

,case_number,filing_quarter_npm,closing_quarter_npm,delta_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,PLT,DEF,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,PROCPROG,DISP,NOJ,JUDGMENT
0,2:11-cv-01037,0.163247,0.226309,0.063062,0.208424,0.222403,0.228644,0.228330,0.227446,0.229692,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),CARROLL,TEXAS INSTRUMENTS INC.,Federal Question,11,27,2,B,830,0,1123,10,14,0,0
1,1:15-cv-01059,0.244910,0.796015,0.551105,-0.038337,0.002570,0.177977,0.783359,-0.105780,-0.160819,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,BLAZER,EBAY INC,Federal Question,11,26,1,B,830,0,1029,10,13,0,0
2,5:11-cv-01349,0.051063,0.047465,-0.003598,0.051389,0.052812,0.056058,0.066061,0.066351,0.069411,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","HOMTEX, INC","LEGGETT & PLATT, INC, ET AL",Federal Question,11,26,5,P,830,0,1043,10,13,0,0
3,2:87-cv-00979,0.010817,0.037153,0.026336,0.011557,0.016528,0.014094,0.034018,0.029158,0.023642,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,THRALL CAR MFG CO,TRINITY INDUSTRIES INC,Federal Question,11,26,2,-8,830,0,88888,5,13,-8,-8
4,1:17-cv-00179,0.208723,0.210924,0.002201,0.207433,0.207305,0.212408,0.211246,0.215499,0.220046,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.","PEN-ONE ACQUISITION GROUP, LLC",APPLE INC.,Federal Question,11,28,1,B,830,0,1097,3,12,0,0


In [211]:
rnd.head()

,case_number,filing_quarter_rnd,closing_quarter_rnd,delta_rnd_sales,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,PLT,DEF,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,PROCPROG,DISP,NOJ,JUDGMENT
0,2:11-cv-01037,0.125210,0.100666,-0.024544,0.118179,0.117259,0.114605,0.098920,0.098242,0.098462,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),CARROLL,TEXAS INSTRUMENTS INC.,Federal Question,11,27,2,B,830,0,1123,10,14,0,0
1,1:15-cv-01059,0.108168,0.125455,0.017288,0.110958,0.111719,0.000000,0.124701,0.127940,0.125441,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,BLAZER,EBAY INC,Federal Question,11,26,1,B,830,0,1029,10,13,0,0
2,5:11-cv-01349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,0.005953,0.006165,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","HOMTEX, INC","LEGGETT & PLATT, INC, ET AL",Federal Question,11,26,5,P,830,0,1043,10,13,0,0
3,2:87-cv-00979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,THRALL CAR MFG CO,TRINITY INDUSTRIES INC,Federal Question,11,26,2,-8,830,0,88888,5,13,-8,-8
4,1:17-cv-00179,0.049904,0.050520,0.000616,0.048885,0.048194,0.046701,0.050661,0.051407,0.052867,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.","PEN-ONE ACQUISITION GROUP, LLC",APPLE INC.,Federal Question,11,28,1,B,830,0,1097,3,12,0,0


# Creating Useful Dummies for the JFC Categorical Data

In [212]:
#using PROCPROG to create disp_before_issue_joined
npm['disp_before_issue_joined'] = 0
rnd['disp_before_issue_joined'] = 0

npm.loc[npm['PROCPROG'].isin([1,2,11,12]), 'disp_before_issue_joined'] = 1
rnd.loc[rnd['PROCPROG'].isin([1,2,11,12]), 'disp_before_issue_joined'] = 1

npm = npm.drop(columns=['PROCPROG'])
rnd = rnd.drop(columns=['PROCPROG'])

In [213]:
#using DISP to create dummies for dissmissals, final judgment and case transfers

#For the npm df
npm['disp_dismissed'] = 0
npm['disp_judgment'] = 0
npm['disp_transfer'] = 0

npm.loc[npm['DISP'].isin([0,1,10,11]), 'disp_transfer'] = 1
npm.loc[npm['DISP'].isin([2,3,12,13,14]), 'disp_dismissed'] = 1
npm.loc[npm['DISP'].isin([4,5,6,7,8,9,15,16,17,18,19,20]), 'disp_judgment'] = 1

#extra dummies for dismissals with settlement or voluntary withdrawl by plaintiff
npm['dismissed_settlement'] = 0
npm['dismissed_voluntary'] = 0

npm.loc[npm['DISP'].isin([13]), 'dismissed_settlement'] = 1
npm.loc[npm['DISP'].isin([12]), 'dismissed_voluntary'] = 1


npm = npm.drop(columns=['DISP'])

#For the rnd df
rnd['disp_dismissed'] = 0
rnd['disp_judgment'] = 0
rnd['disp_transfer'] = 0

rnd.loc[rnd['DISP'].isin([0,1,10,11]), 'disp_transfer'] = 1
rnd.loc[rnd['DISP'].isin([2,3,12,13,14]), 'disp_dismissed'] = 1
rnd.loc[rnd['DISP'].isin([4,5,6,7,8,9,15,16,17,18,19,20]), 'disp_judgment'] = 1

#extra dummies for dismissals with settlement or voluntary withdrawl by plaintiff
rnd['dismissed_settlement'] = 0
rnd['dismissed_voluntary'] = 0

rnd.loc[rnd['DISP'].isin([13]), 'dismissed_settlement'] = 1
rnd.loc[rnd['DISP'].isin([12]), 'dismissed_voluntary'] = 1

rnd = rnd.drop(columns=['DISP'])

In [214]:
#using JUDGMENT to create dummies for who one the cases that went to judgment

#for the npm df
npm['judgment_favor_plt'] = 0
npm['judgment_favor_def'] = 0
npm['judgment_favor_both'] = 0

npm.loc[npm['JUDGMENT'].isin([1]), 'judgment_favor_plt'] = 1
npm.loc[npm['JUDGMENT'].isin([2]), 'judgment_favor_def'] = 1
npm.loc[npm['JUDGMENT'].isin([3]), 'judgment_favor_both'] = 1

npm = npm.drop(columns=['JUDGMENT'])

#for the rnd df
rnd['judgment_favor_plt'] = 0
rnd['judgment_favor_def'] = 0
rnd['judgment_favor_both'] = 0

rnd.loc[rnd['JUDGMENT'].isin([1]), 'judgment_favor_plt'] = 1
rnd.loc[rnd['JUDGMENT'].isin([2]), 'judgment_favor_def'] = 1
rnd.loc[rnd['JUDGMENT'].isin([3]), 'judgment_favor_both'] = 1

rnd = rnd.drop(columns=['JUDGMENT'])

In [215]:
#using NOJ to extract more info about judgment for cases that have judgments

#for the npm df
npm['judgment_monetary_award'] = 0
npm['judgment_injunction'] = 0
npm['judgment_ordered_loss'] = 0

npm.loc[npm['NOJ'].isin([1,2]), 'judgment_monetary_award'] = 1
npm.loc[npm['NOJ'].isin([2]), 'judgment_injunction'] = 1
npm.loc[npm['NOJ'].isin([3,4,5]), 'judgment_monetary_award'] = 1

npm = npm.drop(columns=['NOJ'])

#for the rnd df
rnd['judgment_monetary_award'] = 0
rnd['judgment_injunction'] = 0
rnd['judgment_ordered_loss'] = 0

rnd.loc[rnd['NOJ'].isin([1,2]), 'judgment_monetary_award'] = 1
rnd.loc[rnd['NOJ'].isin([2]), 'judgment_injunction'] = 1
rnd.loc[rnd['NOJ'].isin([3,4,5]), 'judgment_monetary_award'] = 1

rnd = rnd.drop(columns=['NOJ'])

In [216]:
#using jurisdictional basis to create dummies for Fed_Question, US_def and US_plt
npm['FED_QUESTION'] = 0
npm['US_DEF'] = 0
npm['US_PLT'] = 0

npm.loc[npm['jurisdictional_basis'].isin(['Federal Question']), 'FED_QUESTION'] = 1

In [217]:
#filtering out trivial number of 840 trademark infringement cases, focusing on 830 patent infringement
rnd = rnd[rnd['NOS'] == 830]
npm = npm[npm['NOS'] == 830]

#filtering out trivial number of US defendent/plaintiff cases, focusing on Federal Questions
rnd = rnd[rnd['jurisdictional_basis'].isin(['Federal Question'])]
npm = npm[npm['jurisdictional_basis'].isin(['Federal Question'])]

#droping CLASSACT because no classaction suites exist in data
npm = npm.drop(columns=['CLASSACT'])
rnd = rnd.drop(columns=['CLASSACT'])

In [218]:
rnd.head()

,case_number,filing_quarter_rnd,closing_quarter_rnd,delta_rnd_sales,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,PLT,DEF,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_judgment,disp_transfer,dismissed_settlement,dismissed_voluntary,judgment_favor_plt,judgment_favor_def,judgment_favor_both,judgment_monetary_award,judgment_injunction,judgment_ordered_loss
0,2:11-cv-01037,0.125210,0.100666,-0.024544,0.118179,0.117259,0.114605,0.098920,0.098242,0.098462,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),CARROLL,TEXAS INSTRUMENTS INC.,Federal Question,11,27,2,B,830,1123,0,1,0,0,0,0,0,0,0,0,0,0
1,1:15-cv-01059,0.108168,0.125455,0.017288,0.110958,0.111719,0.000000,0.124701,0.127940,0.125441,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,BLAZER,EBAY INC,Federal Question,11,26,1,B,830,1029,0,1,0,0,1,0,0,0,0,0,0,0
2,5:11-cv-01349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,0.005953,0.006165,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","HOMTEX, INC","LEGGETT & PLATT, INC, ET AL",Federal Question,11,26,5,P,830,1043,0,1,0,0,1,0,0,0,0,0,0,0
3,2:87-cv-00979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,THRALL CAR MFG CO,TRINITY INDUSTRIES INC,Federal Question,11,26,2,-8,830,88888,0,1,0,0,1,0,0,0,0,0,0,0
4,1:17-cv-00179,0.049904,0.050520,0.000616,0.048885,0.048194,0.046701,0.050661,0.051407,0.052867,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.","PEN-ONE ACQUISITION GROUP, LLC",APPLE INC.,Federal Question,11,28,1,B,830,1097,0,1,0,0,0,1,0,0,0,0,0,0


In [219]:
npm.head()

,case_number,filing_quarter_npm,closing_quarter_npm,delta_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,PLT,DEF,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_judgment,disp_transfer,dismissed_settlement,dismissed_voluntary,judgment_favor_plt,judgment_favor_def,judgment_favor_both,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,FED_QUESTION,US_DEF,US_PLT
0,2:11-cv-01037,0.163247,0.226309,0.063062,0.208424,0.222403,0.228644,0.228330,0.227446,0.229692,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),CARROLL,TEXAS INSTRUMENTS INC.,Federal Question,11,27,2,B,830,1123,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1:15-cv-01059,0.244910,0.796015,0.551105,-0.038337,0.002570,0.177977,0.783359,-0.105780,-0.160819,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,BLAZER,EBAY INC,Federal Question,11,26,1,B,830,1029,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
2,5:11-cv-01349,0.051063,0.047465,-0.003598,0.051389,0.052812,0.056058,0.066061,0.066351,0.069411,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","HOMTEX, INC","LEGGETT & PLATT, INC, ET AL",Federal Question,11,26,5,P,830,1043,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
3,2:87-cv-00979,0.010817,0.037153,0.026336,0.011557,0.016528,0.014094,0.034018,0.029158,0.023642,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,THRALL CAR MFG CO,TRINITY INDUSTRIES INC,Federal Question,11,26,2,-8,830,88888,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
4,1:17-cv-00179,0.208723,0.210924,0.002201,0.207433,0.207305,0.212408,0.211246,0.215499,0.220046,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.","PEN-ONE ACQUISITION GROUP, LLC",APPLE INC.,Federal Question,11,28,1,B,830,1097,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
